## Load Data

In [1]:
import pandas as pd

In [35]:
def load_data(name, test=False):
    df = pd.read_csv(PATH + '/' + name)
    df.columns = ['index', 'Token', 'Tag']
    if test:
        df.drop('Tag', axis=1, inplace=True)
    df.set_index('index')
    return df

In [36]:
PATH = 'data/'
PATH = PATH.rstrip('/')

# EN Train
en_train = load_data('en_train.csv')

# EN Test
en_test = load_data('en_test.csv', test=True)

# FA Train
fa_train = load_data('fa_train.csv')

# FA TEST
fa_test = load_data('fa_test.csv', test=True)